In [1]:
%load_ext autoreload
%autoreload 2
import torch
import sys
sys.path.append('../scripts/')
from dataset import get_loader, split_datasets, pre_process_data
import pandas as pd
import glob
from loss import calculate_bce
from train import train_script
import torch.nn.functional as F
import torch.nn as nn
import numpy as np
from train import train_script
from model import VGG16
import torchvision.models as models
import torch.optim as optim
from tqdm import tqdm
from metrics import calculate_precision, calculate_recall, calculate_cf
from PIL import Image
import collections

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
train, val, test = split_datasets(pre_process_data('../../dog_cats_data/train/'), 0.1)

In [4]:
print(f'Train set: {len(train)}')
print(f'Val set: {len(val)}')
print(f'Test set: {len(test)}')

Train set: 20250
Val set: 2250
Test set: 2500


In [5]:
train_loader = get_loader(train, 32, 512, True, 6)
val_loader = get_loader(val, 32, 512, True, 6)

In [6]:
# updated_model = models.vgg16(pretrained=False)
# updated_model.classifier[6] = nn.Linear(4096, 1)

In [7]:
# def train_script(model, train_loader, val_loader, epochs, device):
#     dataloaders = {
#         'train': train_loader,
#         'val': val_loader
#     }
#     optimizer = optim.SGD(model.parameters(), lr=0.01)
#     model = model.to(device)
#     for epoch in range(epochs):
#         train_loss, val_loss = 0.0, 0.0
#         train_prec, val_prec = 0.0, 0.0
#         train_rec, val_rec = 0.0, 0.0
#         train_tp, train_fp, train_fn, train_tn = 0.0, 0.0, 0.0, 0.0
#         val_tp, val_fp, val_fn, val_tn = 0.0, 0.0, 0.0, 0.0
#         for phase in ['train', 'val']:
#             if phase == 'train':
#                 model.train()
#             else:
#                 model.eval()
#             running_loss, running_prec, running_rec = 0.0, 0.0, 0.0
#             running_tp, running_fp, running_fn, running_tn = 0.0, 0.0, 0.0, 0.0
#             with tqdm(dataloaders[phase], unit="batch") as tepoch:
#                 for img, labels in tepoch:
#                     img, labels = img.to(device), labels.to(device)
#                     optimizer.zero_grad()
#                     with torch.set_grad_enabled(phase == 'train'):
#                         outputs = model(img)
#                         loss = calculate_bce(outputs, labels)
#                         precision = calculate_precision(outputs, labels)
#                         recall = calculate_recall(outputs, labels)
#                         tp, fn, fp, tn = calculate_cf(outputs, labels)
#                         if phase == 'train':
#                             loss.backward()
#                             optimizer.step()
#                     running_loss += loss.item()
#                     running_prec += precision.item()
#                     running_rec += recall.item()
#                     running_tp += tp.item()
#                     running_fn += fn.item()
#                     running_fp += fp.item()
#                     running_tn += tn.item()
#                     tepoch.set_postfix(loss = loss.item(), precision = precision.item(), recall = recall.item(), tp = tp.item(), fn = fn.item(), fp = fp.item(), tn = tn.item())
#             if phase == 'train':
#                 train_loss = running_loss / len(dataloaders[phase])
#                 train_prec = running_prec / len(dataloaders[phase])
#                 train_rec = running_rec / len(dataloaders[phase])
#                 train_tp = running_tp 
#                 train_fn = running_fn 
#                 train_fp = running_fp 
#                 train_tn = running_tn 
#                 print(f'Train loss: {train_loss}')
#                 print(f'Train prec: {train_prec}')
#                 print(f'Train rec: {train_rec}')
#                 print(f'Train tp: {train_tp}')
#                 print(f'Train fn: {train_fn}')
#                 print(f'Train fp: {train_fp}')
#                 print(f'Train tn: {train_tn}')
#             else:
#                 val_loss = running_loss / len(dataloaders[phase])
#                 val_prec = running_prec / len(dataloaders[phase])
#                 val_rec = running_rec / len(dataloaders[phase])
#                 val_tp = running_tp 
#                 val_fn = running_fn 
#                 val_fp = running_fp 
#                 val_tn = running_tn 
#                 print(f'Val loss: {val_loss}')
#                 print(f'Val prec: {val_prec}')
#                 print(f'Val rec: {val_rec}')
#                 print(f'Val tp: {val_tp}')
#                 print(f'Val fn: {val_fn}')
#                 print(f'Val fp: {val_fp}')
#                 print(f'Val tn: {val_tn}')

In [8]:
# model = models.vgg16(pretrained=False)
# model.classifier[6] = nn.Linear(4096, 1)

In [9]:
train_script(VGG16(), train_loader, val_loader, 50, device)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: yashchks87. Use `wandb login --relogin` to force relogin


100%|██████████| 5/5 [00:03<00:00,  1.49batch/s, fn=19, fp=14, loss=0.965, precision=0.843, recall=0.798, tn=94, tp=75]  
